In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [2]:
import findspark
findspark.init()
import pyspark
conf = (pyspark.SparkConf()
    .setMaster('local')
    .setAppName('pyspark')
    .set("spark.executor.memory", "2g"))
sc = pyspark.SparkContext(conf=conf)

In [4]:
import json
data = []
with open("../cs281_data/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_business.json") as f:
    for l in f:
        data.append(json.loads(l))

In [5]:
business_list = []
    
for b in data:
    d = b.copy()
    
    # Dropping unconsidered features
    d.pop('hours')
    #d.pop('attributes')
    d.pop('neighborhoods')
    d.pop('type')
    d.pop('full_address')
    d.pop('name')    
    
    business_list.append(d)
    
business=pd.DataFrame(business_list)
# Changing the index
business = business.set_index('business_id')

business.head(5)

,attributes,categories,city,latitude,longitude,open,review_count,stars,state
business_id,,,,,,,,,
vcNAWiLM4dR7D2nwwJ7nCA,{u'By Appointment Only': True},"[Doctors, Health & Medical]",Phoenix,33.499313,-111.983758,True,9,3.5,AZ
UsFtqoBl7naz8AVUBZMjQQ,"{u'Happy Hour': True, u'Accepts Credit Cards':...",[Nightlife],Dravosburg,40.350519,-79.886930,True,4,3.5,PA
cE27W9VPgO88Qxe4ol6y_g,{u'Good for Kids': True},"[Active Life, Mini Golf, Golf]",Bethel Park,40.356896,-80.015910,False,5,2.5,PA
HZdLhv6COCleJMo7nPl-RA,{},"[Shopping, Home Services, Internet Service Pro...",Pittsburgh,40.357620,-80.059980,True,3,3.5,PA
mVHrayjG3uZ_RLHkLj-AMg,"{u'Alcohol': u'full_bar', u'Noise Level': u'av...","[Bars, American (New), Nightlife, Lounges, Res...",Braddock,40.408735,-79.866351,True,11,4.5,PA


In [6]:
edinburg = np.array(business.city[business.city=='Edinburgh'].index.values,dtype=str)

In [7]:
import json
review = []
with open("../cs281_data/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_review.json") as f:
    for r in f:
        review.append(json.loads(r))

In [8]:
review[0]

{u'business_id': u'vcNAWiLM4dR7D2nwwJ7nCA',
 u'date': u'2007-05-17',
 u'review_id': u'15SdjuK7DmYqUAj6rjGowg',
 u'stars': 5,
 u'text': u"dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.",
 u'type': u'review',
 u'user_id': u'Xqd0DzHaiyRqVH3WRG7hzg',
 u'votes': {u'cool': 1, u'funny': 0, u'useful': 2}}

In [9]:
rest = np.load('temp/restaurants_index.npy')

In [32]:
business_ids = [r['business_id'] for r in review]

In [43]:
combined_review_dic = dict.fromkeys(list(set(business_ids)))
business_ids_tuples = zip(range(len(business_ids)),business_ids)
for tup in business_ids_tuples:
    if combined_review_dic[tup[1]] is None:
        combined_review_dic[tup[1]] = [review[tup[0]]['text'],[review[tup[0]]['stars']]]
    else:
        combined_review_dic[tup[1]][0] = " ".join([combined_review_dic[tup[1]][0], review[tup[0]]['text']])
        (combined_review_dic[tup[1]][1]).append(review[tup[0]]['stars'])

In [44]:
restaurant_reviews_edinburgh = {}
for r in rest:
    if r in edinburg:
        restaurant_reviews_edinburgh[r] =  combined_review_dic[r]
    

In [45]:
len(restaurant_reviews_edinburgh)

667

In [46]:
for r in restaurant_reviews_edinburgh.keys():
    restaurant_reviews_edinburgh[r][1] = round(np.mean(restaurant_reviews_edinburgh[r][1]))

In [47]:
restaurant_reviews_edinburgh[restaurant_reviews_edinburgh.keys()[0]]

[u"I wandered into town still half asleep before hopping onto a bus up to Aberdeen. My Belly was not happy at the thought of having to wait til reaching the granite city before getting something to eat, so a quick breakfast was what I was looking for. I had walked past this place a few times, it is close to the bus station so I headed straight for it.\n\nI arrived and as it was early morning they were quite busy with takeaways and such like, but it wasn't too busy! I then decided I had enough time to sit and enjoy my breakfast roll rather than walk and eat (that always ends badly for me!!) I grabbed a seat and one of the girls came over to clear the cups and plates off the table. I had decided on what I wanted and sat there for a good ten minutes without anyone coming over to see what I was wanting. Although it does not suggest anywhere that you need to order at the counter, I was a bit peeved that none of the staff thought to come over, or for the girl that was clearing to inform me t

In [15]:
from pattern.en import parse
from pattern.en import pprint
from pattern.vector import stem, PORTER, LEMMA
punctuation = list('.,;:!?()[]{}`''\"@#$^&*+-|=~_')

In [16]:
from sklearn.feature_extraction import text 
stopwords=text.ENGLISH_STOP_WORDS

In [17]:
import re
regex1=re.compile(r"\.{2,}")
regex2=re.compile(r"\-{2,}")

In [37]:
def get_parts(thetext,seperate_w_adj):
    import itertools
    thetext=re.sub(regex1, ' ', thetext)
    thetext=re.sub(regex2, ' ', thetext)
    nouns=[]
    descriptives=[]
    for i,sentence in enumerate(parse(thetext, tokenize=True, lemmata=True).split()):
        nouns.append([])
        descriptives.append([])
        for token in sentence:
            #print token
            if len(token[4]) >0:
                if token[1] in ['JJ', 'JJR', 'JJS']:
                    if token[4] in stopwords or token[4][0] in punctuation or token[4][-1] in punctuation or len(token[4])==1:
                        continue
                    descriptives[i].append(token[4])
                elif token[1] in ['NN', 'NNS']:
                    if token[4] in stopwords or token[4][0] in punctuation or token[4][-1] in punctuation or len(token[4])==1:
                        continue
                    nouns[i].append(token[4])
    out=zip(nouns, descriptives)
    nouns2=[]
    descriptives2=[]
    for n,d in out:
        if len(n)!=0 and len(d)!=0:
            nouns2.append(n)
            descriptives2.append(d)
    if seperate_w_adj:
        return list(itertools.chain.from_iterable(nouns2)), list(itertools.chain.from_iterable(descriptives2))
    else:
        return list(itertools.chain.from_iterable(nouns2+descriptives2))

In [48]:
%%time
edinburgh_reviews_complete = {}
for k in restaurant_reviews_edinburgh.keys():
    edinburgh_reviews_complete[k] = [get_parts(restaurant_reviews_edinburgh[k][0],False),restaurant_reviews_edinburgh[k][1]]

CPU times: user 2min 45s, sys: 1.27 s, total: 2min 46s
Wall time: 2min 47s


In [49]:
edinburgh_reviews_complete[edinburgh_reviews_complete.keys()[0]]

[[u'thought',
  u'granite',
  u'city',
  u'breakfast',
  u'place',
  u'time',
  u'bus',
  u'station',
  u'morning',
  u'takeaway',
  u'minute',
  u'order',
  u'bit',
  u'staff',
  u'girl',
  u'table',
  u'service',
  u'egg',
  u'tattie',
  u'scone',
  u'roll',
  u'minute',
  u'time',
  u'queue',
  u'soup',
  u'board',
  u'lot',
  u'choice',
  u'lunch',
  u'lunch',
  u'rush',
  u'service',
  u'place',
  u'breakfast',
  u'rib',
  u'eye',
  u'steak',
  u'egg',
  u'service',
  u'food',
  u'time',
  u'egg',
  u'toast',
  u'load',
  u'time',
  u'roll',
  u'wifi',
  u'mind',
  u'bite',
  u'portrait',
  u'gallery',
  u'road',
  u'change',
  u'scenery',
  u'inside',
  u'sandwich',
  u'seat',
  u'door',
  u'mistake',
  u'number',
  u'steak',
  u'chicken',
  u'corn',
  u'chowder',
  u'mistake',
  u'number',
  u'reason',
  u'folk',
  u'concept',
  u'door',
  u'dining',
  u'experience',
  u'staff',
  u'door',
  u'time',
  u'mineral',
  u'level',
  u'goodbye',
  u'saline',
  u'intake',
  u'steak',
 

In [50]:
%%time
edinburgh_reviews_wandaj = {}
for k in restaurant_reviews_edinburgh.keys():
    edinburgh_reviews_wandaj[k] = [get_parts(restaurant_reviews_edinburgh[k][0],True),restaurant_reviews_edinburgh[k][1]]

CPU times: user 2min 41s, sys: 1.43 s, total: 2min 42s
Wall time: 2min 43s


In [51]:
edinburgh_reviews_wandaj[edinburgh_reviews_wandaj.keys()[0]]

[([u'thought',
   u'granite',
   u'city',
   u'breakfast',
   u'place',
   u'time',
   u'bus',
   u'station',
   u'morning',
   u'takeaway',
   u'minute',
   u'order',
   u'bit',
   u'staff',
   u'girl',
   u'table',
   u'service',
   u'egg',
   u'tattie',
   u'scone',
   u'roll',
   u'minute',
   u'time',
   u'queue',
   u'soup',
   u'board',
   u'lot',
   u'choice',
   u'lunch',
   u'lunch',
   u'rush',
   u'service',
   u'place',
   u'breakfast',
   u'rib',
   u'eye',
   u'steak',
   u'egg',
   u'service',
   u'food',
   u'time',
   u'egg',
   u'toast',
   u'load',
   u'time',
   u'roll',
   u'wifi',
   u'mind',
   u'bite',
   u'portrait',
   u'gallery',
   u'road',
   u'change',
   u'scenery',
   u'inside',
   u'sandwich',
   u'seat',
   u'door',
   u'mistake',
   u'number',
   u'steak',
   u'chicken',
   u'corn',
   u'chowder',
   u'mistake',
   u'number',
   u'reason',
   u'folk',
   u'concept',
   u'door',
   u'dining',
   u'experience',
   u'staff',
   u'door',
   u'time',
   u

In [52]:
with open('temp/edinburgh_noseparation.json', 'w') as fp:
    json.dump(edinburgh_reviews_complete, fp)
with open('temp/edinburgh_word_adjective.json', 'w') as fp:
    json.dump(edinburgh_reviews_wandaj, fp)